<a href="https://colab.research.google.com/github/MDfarazuddin99/Face_Masked_Recognition/blob/master/VGGFace_Mask_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-6ipqzg3u
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-6ipqzg3u
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=7ecbe92f7c6dcdaba1bd702ba1b847f4a8f3efb77cabfe13356f6a59095ca615
  Stored in directory: /tmp/pip-ephem-wheel-cache-meqzcqjk/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [36]:
pip install keras_vggface

In [0]:
from keras_vggface.vggface import VGGFace

# Based on VGG16 architecture -> old paper(2015)
vggface = VGGFace(model='vgg16') # or VGGFace() as default

In [0]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace

#custom parameters
nb_class = 2
hidden_dim = 512

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3),pooling='avg')
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)


In [0]:
layer_count = 0
for layer in custom_vgg_model.layers:
	layer_count = layer_count+1

for l in range(layer_count-8):
	custom_vgg_model.layers[l].trainable=False
    

In [0]:
custom_vgg_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface import utils
preprocess_input = utils.preprocess_input
datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [76]:
image_size = 224
train_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/Custom/train',
        target_size=(image_size, image_size),
        batch_size=16,
        class_mode='categorical',)

validation_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/Custom/test',
        target_size=(image_size, image_size),
        class_mode='categorical')

custom_vgg_model.fit_generator(
        train_generator,
        validation_data=validation_generator,
        validation_steps=1,epochs=20)

Found 39 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/20
3/3 [==============================] - 1s 189ms/step - loss: 44.8023 - accuracy: 0.6410 - val_loss: 14.6958 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 0s 71ms/step - loss: 4.4880 - accuracy: 0.7949 - val_loss: 74.6913 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 0s 76ms/step - loss: 12.0759 - accuracy: 0.7436 - val_loss: 3.8053 - val_accuracy: 0.9500
Epoch 4/20
3/3 [==============================] - 0s 70ms/step - loss: 4.7749 - accuracy: 0.9487 - val_loss: 8.9575 - val_accuracy: 0.7500
Epoch 5/20
3/3 [==============================] - 0s 92ms/step - loss: 3.8203 - accuracy: 0.9744 - val_loss: 165.1364 - val_accuracy: 0.5000
Epoch 6/20
3/3 [==============================] - 0s 95ms/step - loss: 6.8979 - accuracy: 0.9744 - val_loss: 5.0410 - val_accuracy: 0.9000
Epoch 7/20
3/3 [==============================] - 0s 71ms/step - loss: 5.2362 -

In [0]:
# from IPython.display import display, Javascript
# from google.colab.output import eval_js
# from base64 import b64decode

# def take_photo(filename='photo.jpg', quality=0.8):
#   js = Javascript('''
#     async function takePhoto(quality) {
#       const div = document.createElement('div');
#       const capture = document.createElement('button');
#       capture.textContent = 'Capture';
#       div.appendChild(capture);

#       const video = document.createElement('video');
#       video.style.display = 'block';
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       document.body.appendChild(div);
#       div.appendChild(video);
#       video.srcObject = stream;
#       await video.play();

#       // Resize the output to fit the video element.
#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       // Wait for Capture to be clicked.
#       await new Promise((resolve) => capture.onclick = resolve);

#       const canvas = document.createElement('canvas');
#       canvas.width = video.videoWidth;
#       canvas.height = video.videoHeight;
#       canvas.getContext('2d').drawImage(video, 0, 0);
#       stream.getVideoTracks()[0].stop();
#       div.remove();
#       return canvas.toDataURL('image/jpeg', quality);
#     }
#     ''')
#   display(js)
#   data = eval_js('takePhoto({})'.format(quality))
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return filename

In [0]:
# from IPython.display import Image
# try:
#   filename = take_photo()
#   print('Saved to {}'.format(filename))
  
#   # Show the image which was just taken.
#   display(Image(filename))
# except Exception as err:
#   # Errors will be thrown if the user does not have a webcam or if they do not
#   # grant the page permission to access it.
#   print(str(err))

In [79]:
from keras.preprocessing import image
import numpy as np
test_path = input()
img = image.load_img(test_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=1) # or version=2
preds = custom_vgg_model.predict(x)
print(preds)
if preds[0][0]>preds[0][1]:
    print('Faraz')
else:
    print('Farha')

/content/test_2.jpg
[[0. 1.]]
Farha
